In [1]:
import sys
import os
# Get the absolute path to the directory containing 'notebooks' and 'src'
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:sys.path.append(project_root)


In [3]:
# get the parquet data 
import pandas as pd 

df:pd.DataFrame = pd.read_parquet(path="../data/iitb/raw/train.parquet", engine="pyarrow")
df

,en,hi
0,Give your application an accessibility workout,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1,Accerciser Accessibility Explorer,एक्सेर्साइसर पहुंचनीयता अन्वेषक
2,The default plugin layout for the bottom panel,निचले पटल के लिए डिफोल्ट प्लग-इन खाका
3,The default plugin layout for the top panel,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
4,A list of plugins that are disabled by default,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...
...,...,...
1659078,"The Prime Minister, Shri Narendra Modi has con...",प्रधानमंत्री श्री नरेन्द्र मोदी ने बिहार के मु...
1659079,"In a tweet, the Prime Minister said, congratul...","एक ट्वीट में प्रधानमंत्री ने कहा, बिहार के मुख..."
1659080,I also congratulate all those who took oath as...,"मैं उन सभी को भी बधाई देता हूं, जिन्होंने बिहा..."
1659081,The NDA family will work together for the prog...,एनडीए परिवार बिहार की प्रगति के लिए साथ मिलकर ...


In [4]:
df.tail(10)

,en,hi
1659073,The Commission had submitted its report to the...,आयोग ने अपनी रिपोर्ट 4 नवंबर 2020 को भारत के र...
1659074,Chairman Shri N K Singh along with Members of ...,अध्यक्ष श्री एन. के. सिंह तथा आयोग के सदस्य श्...
1659075,The Commission will present its Report to the ...,आयोग कल केंद्रीय वित्त मंत्री को अपनी रिपोर्ट ...
1659076,The Report will be placed on the Table of the ...,संविधान के तहत निर्धारित प्रक्रिया के अनुरूप इ...
1659077,PM congratulates Shri Nitish Kumar on taking o...,प्रधानमंत्री ने बिहार के मुख्यमंत्री पद की शपथ...
1659078,"The Prime Minister, Shri Narendra Modi has con...",प्रधानमंत्री श्री नरेन्द्र मोदी ने बिहार के मु...
1659079,"In a tweet, the Prime Minister said, congratul...","एक ट्वीट में प्रधानमंत्री ने कहा, बिहार के मुख..."
1659080,I also congratulate all those who took oath as...,"मैं उन सभी को भी बधाई देता हूं, जिन्होंने बिहा..."
1659081,The NDA family will work together for the prog...,एनडीए परिवार बिहार की प्रगति के लिए साथ मिलकर ...
1659082,I assure all possible support from the Centre ...,बिहार के कल्याण के लिए केंद्र की ओर से हरसंभव ...


In [10]:
# we can build shared vocab for both english and hindi sentences.
from tokenizers import Tokenizer
from tokenizers import models, pre_tokenizers, trainers

tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

bpe_trainer = trainers.BpeTrainer(vocab_size=5334,special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"])
tokenizer.train_from_iterator(df["en"].tolist() + df["hi"].tolist(), trainer=bpe_trainer)

In [11]:
tokenizer

Tokenizer(version="1.0", truncation=None, padding=None, added_tokens=[{"id":0, "content":"[UNK]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":1, "content":"[PAD]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":2, "content":"[SOS]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":3, "content":"[EOS]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}], normalizer=None, pre_tokenizer=Whitespace(), post_processor=None, decoder=None, model=BPE(dropout=None, unk_token="[UNK]", continuing_subword_prefix=None, end_of_word_suffix=None, fuse_unk=False, byte_fallback=False, ignore_merges=False, vocab={"[UNK]":0, "[PAD]":1, "[SOS]":2, "[EOS]":3, "":4, "":5, "!":6, """:7, "#":8, "$":9, "%":10, "&":11, "'":12, "(":13, ")":14, "*":15, "+":16, ",":17, "-":18, ".":19, "/":20, "0":21, "1":22, "2":23, "3":24, "4":25

In [12]:
tokenizer.save("demo_tokenizer.json")

In [15]:
tokenizer.encode("pranab mukherjee").ids

[5307, 5306]

In [16]:
tokenizer.decode([5307, 5306])

'pranab mukherjee'

In [17]:
tokenizer.encode("My name is Anvesh Dange").ids

[2580, 1971, 921, 1857, 1868, 77, 41, 1800]

In [19]:
tokenizer.decode([2580, 1971, 921, 1857, 1868, 77, 41, 1800])

'My name is An ves h D ange'

In [34]:
from tokenizers import Tokenizer
from tokenizers import models
from tokenizers import pre_tokenizers
from tokenizers import trainers
from tokenizers import decoders 

new_tokenizer = Tokenizer(models.BPE(
    unk_token="[UNK]",
    end_of_word_suffix="</w>")
)

new_tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

bpe_trainer = trainers.BpeTrainer(
    vocab_size=5334,
    end_of_word_suffix="</w>", 
    special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"]
)

new_tokenizer.train_from_iterator(df["en"].tolist() + df["hi"].tolist(), trainer=bpe_trainer)
new_tokenizer.decoder = decoders.BPEDecoder(
    suffix="</w>"
)


In [35]:
new_tokenizer.save("new_tokenizer.json")

In [42]:
# Input String
text = "My name is Anvesh Dange"

print("Tokenization:")
print(f"\"{text}\" -----> {new_tokenizer.encode(text).ids}")
print("\nDetokenization:")
print(f"{new_tokenizer.encode(text).ids} -----> \"{new_tokenizer.decode(new_tokenizer.encode(text).ids)}\"")

# Input String
text = "मेरा नाम अन्वेष डांगे है"

print("\nTokenization:")
print(f"\"{text}\" -----> {new_tokenizer.encode(text).ids}")
print("\nDetokenization:")
print(f"{new_tokenizer.encode(text).ids} -----> \"{new_tokenizer.decode(new_tokenizer.encode(text).ids)}\"")

Tokenization:
"My name is Anvesh Dange" -----> [3640, 2732, 1547, 1819, 91, 1585, 910, 41, 2797]

Detokenization:
[3640, 2732, 1547, 1819, 91, 1585, 910, 41, 2797] -----> "My name is Anvesh Dange"

Tokenization:
"मेरा नाम अन्वेष डांगे है" -----> [3937, 2442, 2023, 1862, 990, 2148, 2009, 1528]

Detokenization:
[3937, 2442, 2023, 1862, 990, 2148, 2009, 1528] -----> "मेरा नाम अन्वेष डांगे है"


In [44]:
entire_data: list[str] = df["en"].tolist() + df["hi"].tolist()

In [45]:
len(entire_data)

3318166

In [65]:
for i in range(1, 6) :print(entire_data[int(f"-{i}")]) 

बिहार के कल्याण के लिए केंद्र की ओर से हरसंभव सहयोग का आश्वासन देता हूं।
एनडीए परिवार बिहार की प्रगति के लिए साथ मिलकर काम करेगा।
मैं उन सभी को भी बधाई देता हूं, जिन्होंने बिहार सरकार में मंत्री पद की शपथ ली है।
एक ट्वीट में प्रधानमंत्री ने कहा, बिहार के मुख्यमंत्री के रूप में शपथ लेने पर  नीतीश कुमार जी को बधाई।
प्रधानमंत्री श्री नरेन्द्र मोदी ने बिहार के मुख्यमंत्री के रूप में शपथ लेने पर श्री नीतीश कुमार को बधाई दी है।


In [66]:
for i in range(1, 6) :print(entire_data[i]) 

Accerciser Accessibility Explorer
The default plugin layout for the bottom panel
The default plugin layout for the top panel
A list of plugins that are disabled by default
Highlight duration


In [67]:
new_tokenizer.from_file(str("new_tokenizer.json"))

Tokenizer(version="1.0", truncation=None, padding=None, added_tokens=[{"id":0, "content":"[UNK]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":1, "content":"[PAD]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":2, "content":"[SOS]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}, {"id":3, "content":"[EOS]", "single_word":False, "lstrip":False, "rstrip":False, "normalized":False, "special":True}], normalizer=None, pre_tokenizer=Whitespace(), post_processor=None, decoder=BPEDecoder(suffix="</w>"), model=BPE(dropout=None, unk_token="[UNK]", continuing_subword_prefix=None, end_of_word_suffix="</w>", fuse_unk=False, byte_fallback=False, ignore_merges=False, vocab={"[UNK]":0, "[PAD]":1, "[SOS]":2, "[EOS]":3, "":4, "":5, "!":6, """:7, "#":8, "$":9, "%":10, "&":11, "'":12, "(":13, ")":14, "*":15, "+":16, ",":17, "-":18, ".":19, "/":20, "0":21, "1":22,

In [70]:
p = new_tokenizer.get_vocab()

In [71]:
p

{'कं': 2545,
 'ill</w>': 1727,
 'झे</w>': 2338,
 'य़</w>': 1257,
 'These</w>': 3719,
 'section</w>': 2726,
 'fam': 3193,
 'होती</w>': 2408,
 '\x86</w>': 1269,
 'पी</w>': 2987,
 'fl': 2739,
 'ang': 2193,
 'ps</w>': 2791,
 'N</w>': 1013,
 'aff': 3459,
 'blic</w>': 2959,
 'श्री</w>': 2930,
 'लक</w>': 4693,
 'पौ': 5086,
 'फो': 3600,
 '[</w>': 1106,
 'from</w>': 1743,
 'commun': 3443,
 'fol': 2287,
 'बहु': 2039,
 'control</w>': 4994,
 'structure</w>': 4503,
 'what</w>': 2125,
 'ο': 333,
 'avail': 3697,
 'भेजा</w>': 4851,
 '⅓': 614,
 '國</w>': 1368,
 'मान': 2065,
 'उसने</w>': 2567,
 '編': 853,
 'â</w>': 998,
 'elop': 2472,
 'stry</w>': 3201,
 'उनको</w>': 3251,
 '國': 770,
 'ĭ</w>': 1410,
 'exist': 4136,
 'hea': 2466,
 'ô': 205,
 'ins</w>': 3455,
 'ज़': 1705,
 'essing</w>': 5182,
 'uses</w>': 5328,
 '参': 763,
 'along</w>': 4978,
 'conn': 4340,
 'फि': 1870,
 'rate</w>': 4660,
 'नल</w>': 4253,
 'का': 1567,
 'wn</w>': 1964,
 'deeds</w>': 4743,
 'It</w>': 2055,
 'भवन</w>': 3659,
 'ű': 270,
 'ress</w>

In [77]:
p = Tokenizer(models.BPE())
p.from_file("../tokeniser/bpe.json")
p

Tokenizer(version="1.0", truncation=None, padding=None, added_tokens=[], normalizer=None, pre_tokenizer=None, post_processor=None, decoder=None, model=BPE(dropout=None, unk_token=None, continuing_subword_prefix=None, end_of_word_suffix=None, fuse_unk=False, byte_fallback=False, ignore_merges=False, vocab={}, merges=[]))

In [78]:
p.get_vocab()

{}

In [79]:
p.encode("I am anvesh").ids

[]

In [80]:
new_tokenizer.get_vocab()

{'Wh': 2132,
 'जीवन</w>': 2662,
 '*</w>': 1082,
 'निर्धार': 2855,
 'श्': 1697,
 'Tr': 5214,
 'व्यापार</w>': 4566,
 'equ': 2758,
 '⅔</w>': 1435,
 'चौ': 3823,
 'Θ': 314,
 'prac': 4563,
 'eafter</w>': 5220,
 'शरीर</w>': 4662,
 'strict</w>': 5076,
 'कों</w>': 2124,
 'ever</w>': 2084,
 '゚': 690,
 'University</w>': 5278,
 'English</w>': 5256,
 'वों</w>': 3878,
 'धि</w>': 2156,
 'g</w>': 936,
 'ध्यम</w>': 3582,
 'idence</w>': 5112,
 'क्या</w>': 1977,
 'उप': 1805,
 'फ़ाइल</w>': 3248,
 'Mo': 2422,
 '択': 804,
 'act</w>': 2811,
 '￼</w>': 1184,
 '€</w>': 1277,
 'man': 2263,
 'є</w>': 1110,
 'आय</w>': 3411,
 'な</w>': 1362,
 'एव': 2362,
 'Those</w>': 4550,
 'ध्व': 5299,
 '?': 36,
 'ilities</w>': 4776,
 'सूच': 2185,
 '16</w>': 4413,
 'щ': 394,
 'विभा': 4977,
 'सरकार</w>': 2504,
 'पश्चि': 4384,
 'A': 38,
 'फर': 4328,
 'clear</w>': 3836,
 'जब': 3574,
 'कार्': 1881,
 'rom</w>': 1734,
 'respon': 3317,
 'औ': 457,
 'अधीन</w>': 3271,
 'डल</w>': 4188,
 '⌘</w>': 1150,
 'प्रश्': 4675,
 'Ch': 2305,
 'Co': 4557,

In [82]:
from tokenizers import Tokenizer

p = Tokenizer.from_file("new_tokenizer.json") 

In [83]:
p.get_vocab()

{'sel': 2002,
 'surely</w>': 3817,
 'हि': 1742,
 'भूमिका</w>': 5055,
 'ged</w>': 3929,
 'many</w>': 2754,
 'ना</w>': 1582,
 'Д</w>': 1332,
 'त्र': 1730,
 'चा</w>': 2500,
 'प्त</w>': 2088,
 'ध्य</w>': 4112,
 'त्रु': 4052,
 'ful</w>': 2236,
 'み': 679,
 'λ': 330,
 'him': 4298,
 'those</w>': 2042,
 'Rashtra': 4497,
 'शा': 1792,
 'सहा': 2653,
 '.': 19,
 '失': 777,
 'जाओ</w>': 4983,
 'त्व': 2606,
 'ource</w>': 4451,
 'हे': 2832,
 'बल्': 2939,
 '致': 861,
 'Engl': 5040,
 'जी</w>': 2030,
 'सदस्': 3180,
 'ems</w>': 3121,
 'प्रि': 3996,
 '-': 18,
 'ं': 440,
 'end</w>': 2315,
 'ti</w>': 4446,
 'desh</w>': 4765,
 'झे</w>': 2338,
 'वास्तव</w>': 4391,
 'money</w>': 5245,
 'Amer': 4433,
 '曜': 818,
 'father</w>': 5165,
 '加</w>': 1293,
 '成</w>': 1366,
 'बा</w>': 3571,
 'õ</w>': 1307,
 'ţ</w>': 1042,
 'he': 4399,
 'Col': 4800,
 'del': 3972,
 '⏎</w>': 1507,
 'पं': 2590,
 'स्ता': 2298,
 'ffic': 2441,
 'present</w>': 3806,
 'वर्तमान</w>': 4431,
 'dition</w>': 3733,
 'ली': 2527,
 '⁷': 606,
 'स</w>': 940,
 'ad

In [86]:
p.encode("I am anvesh").ids

[935, 2274, 1517, 91, 1585, 910]